<br>
<center>

<img src='logo.png' alt='logo udea' height=200>

</center>

<center>

# **1. PyLace** 

## En el siguiente notebook se describira la funcionalidad de la libreria Lace mediante ejemplos basicos y se explicaran algunas consideraciones, asi como una breve descripcion de cada metodo que se considere importante / interesante / que pueda aportar al desarrollo del entendimiento 

##### Lace es una libreria escrita en rust con con una interfaz de uso para python. A diferencia de los metodos tradicionales de machine learning, los cuales aprenden acerca de funciones que asignan entradas a salidas, Lace aprende sobre distribuciones de probabilidades conjuntas relacionadas al dataset empleado. Esta importante caracteristica le permite a los usuarios:

<style>

  .grid {
    display: grid;
    grid-template-columns: repeat(2,1fr);
    gap: 2rem;
  }

</style>

<br>

<div class='grid'>

  <div>
  
  ##### **Predicción y Computación de Probabilidades:**

  ##### -Calcular la probabilidad de cualquier conjunto de características condicionado por otras características (independientemente de la cantidad).

  ##### **Cuantificación de la Incertidumbre:**

  ##### -Identificar, cuantificar y atribuir la incertidumbre proveniente de la varianza en los datos, la incertidumbre epistemológica del modelo y las características faltantes.

  ##### **Análisis de Dependencia entre Variables:**

  ##### -Determinar qué variables predicen a otras.

  ##### **Análisis de Similitud:**

  ##### -Identificar qué registros/filas son similares entre sí en general o en un contexto específico.

  ##### **Generación de Datos Sintéticos:**

  ##### -Simular y manipular datos sintéticos.

  </div>

  <div>
  
  ##### **Manejo de Datos Faltantes:**

  ##### -Trabajar de forma nativa con datos faltantes y realizar inferencias sobre la falta de información (incluso cuando falten datos no aleatoriamente).

  ##### **Flexibilidad en Tipos de Datos:**

  ##### -Trabajar con datos continuos y categóricos de forma nativa, sin necesidad de transformación.

  ##### **Detección de Anomalías:**

  ##### -Identificar anomalías, errores e inconsistencias dentro de los datos.

  ##### **Edición y Manipulación de Datos:**

  ##### -Editar, rellenar y anexar datos sin necesidad de reentrenamiento del modelo.
    
  </div>

</div>





In [73]:
#Se importan las librerias necesarias
import pandas as pd
import lace
from lace.examples import ExamplePaths

import warnings
warnings.filterwarnings("ignore")

In [23]:
path = ExamplePaths('animals').data
df = pd.read_csv(path)
df.set_index('id', inplace=True)

In [96]:
#Se crea un "motor" a partir del dataframe cargado
codebook = lace.Codebook.from_df(name='engine', df=df)

engine = lace.Engine.from_df(df, codebook=codebook)

# Se ajusta un modelo al dataset a lo largo de 5000 iteraciones de ajuste. 
engine.update(6000)

# Se muestra la estructura estadística de los datos, es decir, que características son dependientes (predictivas) entre sí
engine.clustermap("depprob", zmin=0, zmax=1).figure.show()

  2%|▏         | 834/48000 [00:00<00:05, 8285.07it/s]

100%|██████████| 48000/48000 [00:07<00:00, 6266.01it/s]


### Como consideraciones a la hora de ajustar el "modelo" es que a mayor iteraciones ingresados en el metodo **update** mejor se ajustara. Se puede temer que entre mas iteraciones se hagan para el ajuste haya una gran probabilidad de caer en el overfitting. Como Lace utiliza para su ajuste el algortimo mediante la cadena de Markov Monte Carlo es muy poco probable el overfitting, a diferencia de lo que puede ocurrir en un algoritmo tradicional de machine o deep learning    

<center>
<img src='state-fitting.gif' alt='gif'>

### En el gif anterior se puede observar como la libreria va haciendo sus particiones al dataset en sus posibles vistas y categorias. Estas particiones dependen de la cantidad de iteraciones que se le asignen al metodo **update**, a mayor cantidad de iteraciones mayor capacidad de prediccion

<center>

# **Predicciones**

### Ahora que se conoce cuales columnas son predictivas unas con las otras (heatmap anterior), se pone a prueba el poder de prediccion de la libreria para corroborar que sea fiel a la intuicion mostrada  

In [97]:
# Prediccion si un animal nada. Solo un animal (Que es la informacion general del dataset), no si este tiene alas, o cola, o si es un pez, etc.  

engine.predict("swims")

(0, 0.044012897168884686)

### El primer elemento de la tupla es la prediccion realizada. Se puede observa que la libreria predice que los animales no nadan (la mayoria del dataset no nadan). El segundo elemento es la incertidumbre. Este es un numero entre 0 y 1 que representa el desacuerdo entre los estados. Si la incertidumbre es muy cercana a 0 la mayoria de los estados estan deacuerdo en como el modelo predijo, y si es muy cercano a 1 la mayoria de los estados estan en desacuerdo con la prediccion del modelo

### Para dar mayor especificidad a las predicciones se le pueden agregar condiciones

In [98]:
engine.predict("swims", given={'flippers': 1, 'water': 1})

(1, 0.03186696777036702)

### Aca se puede observar una prediccion verdadera, es decir. Si el animal es de agua y tiene aletas nada. Esto es una inferencia basica que no se sale del sentido comun, pero se comprueba que la libreria hace un buen trabajo en predecir. Esto ademas se puede corroborar en la incertidumbre, que al igual que en la prediccion sin filtros, se obtiene un valor muy bajo 

### Como ultima prueba se puede intentar llevar al metodo predict a un punto absurdo y observar su prediccion, para asi corroborar por completo el buen funcionamiento 

In [101]:
engine.predict("swims", given={'water': 0, 'flippers': 1})

(0, 0.3286822238457029)

### Si se toma un intervalo de confianza del 5% se puede observar que lo supera con creces, es decir, la incertidumbre es lo suficientemente grande para rechazar la prediccion. Con esto se observa que al llevar a un caso absurdo la libreria responde con un buen funcionamiento, aclarando que el posible error de prediccion es relativamente alto. Esto se puede observar tambien en el calculo de las probabilidades para ver que esta sucediendo

In [110]:
engine.logp(
  pd.Series([0,1], name='swims'),
  given={'flippers': 1, 'water': 0}
).exp()

logp
f64
0.541476
0.458524
